# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 6 2022 8:46AM,238825,10,Enova-10081,Emerginnova,Released
1,Jun 6 2022 8:46AM,238825,10,Enova-10083,Emerginnova,Released
2,Jun 6 2022 8:46AM,238824,10,0085740238,ISDIN Corporation,Released
3,Jun 6 2022 8:46AM,238824,10,0085740236,ISDIN Corporation,Released
4,Jun 6 2022 8:46AM,238824,10,0085740237,ISDIN Corporation,Released
5,Jun 6 2022 8:46AM,238824,10,0085740256,ISDIN Corporation,Released
6,Jun 6 2022 8:46AM,238824,10,0085740233,ISDIN Corporation,Released
7,Jun 6 2022 8:46AM,238824,10,0085740070,ISDIN Corporation,Released
8,Jun 6 2022 8:46AM,238824,10,0085740069,ISDIN Corporation,Released
9,Jun 6 2022 8:46AM,238824,10,0085740071,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
5,238821,Released,2
6,238822,Released,14
7,238823,Released,29
8,238824,Released,58
9,238825,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
238821,NaN,2.0
238822,NaN,14.0
238823,NaN,29.0
238824,NaN,58.0
238825,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
238818,0.0,1.0
238819,6.0,23.0
238820,1.0,13.0
238821,0.0,2.0
238822,0.0,14.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
238818,0,1
238819,6,23
238820,1,13
238821,0,2
238822,0,14


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,238818,0,1
1,238819,6,23
2,238820,1,13
3,238821,0,2
4,238822,0,14


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,238818,,1
1,238819,6,23
2,238820,1,13
3,238821,,2
4,238822,,14


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 6 2022 8:46AM,238825,10,Emerginnova
2,Jun 6 2022 8:46AM,238824,10,ISDIN Corporation
60,Jun 6 2022 8:43AM,238823,10,ISDIN Corporation
89,Jun 6 2022 8:32AM,238822,10,ISDIN Corporation
103,Jun 6 2022 8:27AM,238821,19,ACG North America LLC
105,Jun 6 2022 7:55AM,238820,19,"AdvaGen Pharma, Ltd"
119,Jun 6 2022 7:49AM,238819,19,"Innogenix, LLC"
148,Jun 6 2022 7:30AM,238818,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 6 2022 8:46AM,238825,10,Emerginnova,,2
1,Jun 6 2022 8:46AM,238824,10,ISDIN Corporation,,58
2,Jun 6 2022 8:43AM,238823,10,ISDIN Corporation,,29
3,Jun 6 2022 8:32AM,238822,10,ISDIN Corporation,,14
4,Jun 6 2022 8:27AM,238821,19,ACG North America LLC,,2
5,Jun 6 2022 7:55AM,238820,19,"AdvaGen Pharma, Ltd",1,13
6,Jun 6 2022 7:49AM,238819,19,"Innogenix, LLC",6,23
7,Jun 6 2022 7:30AM,238818,22,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 6 2022 8:46AM,238825,10,Emerginnova,2,
1,Jun 6 2022 8:46AM,238824,10,ISDIN Corporation,58,
2,Jun 6 2022 8:43AM,238823,10,ISDIN Corporation,29,
3,Jun 6 2022 8:32AM,238822,10,ISDIN Corporation,14,
4,Jun 6 2022 8:27AM,238821,19,ACG North America LLC,2,
5,Jun 6 2022 7:55AM,238820,19,"AdvaGen Pharma, Ltd",13,1
6,Jun 6 2022 7:49AM,238819,19,"Innogenix, LLC",23,6
7,Jun 6 2022 7:30AM,238818,22,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 6 2022 8:46AM,238825,10,Emerginnova,2,
1,Jun 6 2022 8:46AM,238824,10,ISDIN Corporation,58,
2,Jun 6 2022 8:43AM,238823,10,ISDIN Corporation,29,
3,Jun 6 2022 8:32AM,238822,10,ISDIN Corporation,14,
4,Jun 6 2022 8:27AM,238821,19,ACG North America LLC,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 6 2022 8:46AM,238825,10,Emerginnova,2,NaN
1,Jun 6 2022 8:46AM,238824,10,ISDIN Corporation,58,NaN
2,Jun 6 2022 8:43AM,238823,10,ISDIN Corporation,29,NaN
3,Jun 6 2022 8:32AM,238822,10,ISDIN Corporation,14,NaN
4,Jun 6 2022 8:27AM,238821,19,ACG North America LLC,2,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 6 2022 8:46AM,238825,10,Emerginnova,2,0.0
1,Jun 6 2022 8:46AM,238824,10,ISDIN Corporation,58,0.0
2,Jun 6 2022 8:43AM,238823,10,ISDIN Corporation,29,0.0
3,Jun 6 2022 8:32AM,238822,10,ISDIN Corporation,14,0.0
4,Jun 6 2022 8:27AM,238821,19,ACG North America LLC,2,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,955294,103,0.0
19,716460,38,7.0
22,238818,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,955294,103,0.0
1,19,716460,38,7.0
2,22,238818,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,103,0.0
1,19,38,7.0
2,22,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,103.0
1,19,Released,38.0
2,22,Released,1.0
3,10,Executing,0.0
4,19,Executing,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,22
Status,,,
Executing,0.0,7.0,0.0
Released,103.0,38.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,22
0,Executing,0.0,7.0,0.0
1,Released,103.0,38.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,22
0,Executing,0.0,7.0,0.0
1,Released,103.0,38.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()